## Azure ML - Sample Pipeline for File Dataset Creation/Consumption
This notebook demonstrates creation and execution of an Azure ML pipeline designed to create pandas dataframes filled with random data and save these as CSVs to a File Dataset. This File Dataset is subsequently consumed both as a mount and a download in downstream steps.  

### Import Required Packages

In [ ]:
from azureml.core import Workspace, Experiment, Datastore, Environment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.pipeline.core import Pipeline, PipelineParameter, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineParameter, PipelineData, PipelineEndpoint
from azureml.data.output_dataset_config import OutputTabularDatasetConfig, OutputDatasetConfig, OutputFileDatasetConfig

### Connect to Azure ML Workspace, Provision Compute Resources, and get References to Datastores
Connect to workspace using config associated config file. Get a reference to you pre-existing AML compute cluster or provision a new cluster to facilitate processing. Finally, get references to your default blob datastore.

In [ ]:
#Connect to AML Workspace
ws = Workspace.from_config()

# #Select AML Compute Cluster
cpu_cluster_name = 'cpucluster'

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found an existing cluster, using it instead.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_V2',
                                                           min_nodes=0,
                                                           max_nodes=1)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    cpu_cluster.wait_for_completion(show_output=True)
    
#Get default datastore
default_ds = ws.get_default_datastore()

 ### Create Run Configuration
The `RunConfiguration` defines the environment used across all python steps. You can optionally add additional conda or pip packages to be added to your environment. [More details here](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.conda_dependencies.condadependencies?view=azure-ml-py).

In [ ]:
run_config = RunConfiguration()
run_config.docker.use_docker = True
run_config.docker.shm_size='10g'
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas'])

### Define Output Datasets
Below we define the configuration for the `FileDataset` that will be passed between steps in our pipeline. 

In [ ]:
sample_file_dataset = OutputFileDatasetConfig(name='sample_file_dataset', destination=(default_ds, 'sample_file_dataset/{run-id}')).register_on_complete(name='sample_file_dataset')

### Define Pipeline Steps
The pipeline below consists of steps to gather and register data from a remote source, a scoring step where the registered model is used to make predictions on loaded, and a data publish step where scored data can be exported to a remote data source. All of the PythonScriptSteps have a corresponding *.py file which is referenced in the step arguments. Also, any PipelineParameters defined above can be passed to and consumed within these steps.

In [ ]:
register_data_step = PythonScriptStep(
    name='Register File Dataset',
    script_name='register_file_dataset.py',
    arguments=[
        '--sample_file_dataset', sample_file_dataset
    ],
    outputs=[sample_file_dataset],
    compute_target=cpu_cluster,
    source_directory='./pipeline_step_scripts',
    allow_reuse=False,
    runconfig=run_config
)

consume_data_as_download_step = PythonScriptStep(
    name='Consume File Dataset as Download',
    script_name='consume_file_dataset_as_download.py',
    arguments=[
        '--local_download_dir', './tmpdir'
    ],
    inputs=[sample_file_dataset.as_input(name='sample_file_dataset').as_download('./tmpdir')],
    outputs=[],
    compute_target=cpu_cluster,
    source_directory='./pipeline_step_scripts',
    allow_reuse=False,
    runconfig=run_config
)

consume_data_as_mount_step = PythonScriptStep(
    name='Consume File Dataset as Mount',
    script_name='consume_file_dataset_as_mount.py',
    arguments=[],
    inputs=[sample_file_dataset.as_input(name='sample_file_dataset').as_mount()],
    outputs=[],
    compute_target=cpu_cluster,
    source_directory='./pipeline_step_scripts',
    allow_reuse=False,
    runconfig=run_config
)

### Create Pipeline
Create an Azure ML Pipeline by specifying the steps to be executed. Note: based on the dataset dependencies between steps, exection occurs logically such that no step will execute unless all of the necessary input datasets have been generated.

In [ ]:
pipeline = Pipeline(workspace=ws, steps=[register_data_step, consume_data_as_download_step, consume_data_as_mount_step])

### Create Experiment and Run Pipeline
Define a new experiment (logical container for pipeline runs) and execute the pipeline. You can modify the values of pipeline parameters here when submitting a new run.

In [ ]:
experiment = Experiment(ws, 'file_dataset_testing')
run = experiment.submit(pipeline)
run.wait_for_completion(show_output=True)